In [14]:
import pandas as pd
import multiprocessing as mp
import itertools
import networkx as nx
import sys
import pickle
import time  
from tqdm import tqdm

In [15]:
def mainp01(cores):
    global global_string_G  
    start_time = time.time()  

    DIP_addr = "Data/DIP/CPTAC_SIP_deg.txt"
    string_addr = "Data/Human_network/Human_Interactome2022.txt"

    DIP_df = pd.read_csv(DIP_addr, sep='\t')
  

    DIP_list = DIP_df['StringGene'].to_list()
    

    with open(string_addr) as network_f:
        string_network_edges = [x.strip().split('\t') for x in network_f.readlines()]

    global_string_G = nx.Graph(string_network_edges)
    
    
    TNBC_sub_structures = list(set(DIP_df['Bait'].to_list()))

    sub_structure_result = dict()
    pool = mp.Pool(processes=int(cores))
    
    for sub_structure in TNBC_sub_structures:
        DIP_sub_list = DIP_df[DIP_df['Bait'] == sub_structure]['StringGene'].to_list()
        DIP_pairs = list(itertools.product(DIP_sub_list, DIP_list))
        
        
        DIP_pairs_filtered = [
            pair for pair in DIP_pairs 
            if pair[0] in global_string_G.nodes and pair[1] in global_string_G.nodes
        ]


        SP_result = list(tqdm(pool.imap(process_pair, DIP_pairs_filtered), total=len(DIP_pairs_filtered)))

        sub_structure_result[sub_structure] = SP_result
    pool.close()
    pool.join()
    

    all_edges = [edge for edges in sub_structure_result.values() for edge_list in edges for edge in edge_list if edge]

    SIG_G = nx.Graph()
    SIG_G.add_edges_from(all_edges)

    final_nodes = set(DIP_list)
    filtered_edges = [(u, v) for u, v in SIG_G.edges() if u in final_nodes and v in final_nodes]
    SIG_G = nx.Graph()
    SIG_G.add_edges_from(filtered_edges)

    save_obj(SIG_G, "Result/Network/CPTAC_SIP_All_Structure_All_Shortest_Paths_graph")

    end_time = time.time()  
    total_time = end_time - start_time 



In [ ]:
mainp01(cores=64)

In [ ]:
SIG_G = load_obj("Result/Network/metadata_M13_All_Structure_All_Shortest_Paths_graph")